### Import Libraries

In [1]:
import numpy as np
import pandas as pd
import BPt as bp
import os

from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)


c:\Users\simon\AppData\Local\Programs\Python\Python37\lib\site-packages\nilearn\datasets\__init__.py:96: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  "Numpy arrays.", FutureWarning)


###  Define helper functions and path of data

In [22]:
path = "../ABCD 3.0/"
read_file = lambda file_name: pd.read_csv(file_name, delimiter='\t', skiprows=[1], index_col='subjectkey', low_memory=False)
baseline = lambda df: df[df['eventname']=='baseline_year_1_arm_1']
followup = lambda df: df[df['eventname']=='1_year_follow_up_y_arm_1']

###  Covariate Files

In [3]:
### Calculating BMI
ant01 = followup(read_file(path+'abcd_ant01.txt'))[['anthroweightcalc','anthroheightcalc']]
m = ant01['anthroweightcalc']
h2 = ant01['anthroheightcalc']**2
ant01['bmi'] = (m/h2)*703
ant01 = ant01[['bmi']]
print('ant01 shape:', ant01.shape)

ant01 shape: (4951, 1)


In [4]:
### Puberty Score
ppdms01 =baseline(read_file(path+'abcd_ppdms01.txt'))
ppdms01 = ppdms01.replace(999.0, 0)
ppdms01 = ppdms01.replace(np.nan, 0)
ppdms01['pubertal_score'] = ppdms01.apply(lambda x : x['pds_1_p'] + x['pds_2_p'] + x['pds_3_p'] + x['pds_m4_p'] + x['pds_m5_p']  if (x['pubertal_sex_p']==1.0) else x['pds_1_p'] + x['pds_2_p'] + x['pds_3_p'] + x['pds_f4_p'] + x['pds_f5b_p'], axis=1, result_type='reduce')
ppdms01 = ppdms01[['pubertal_score']].astype('int')
ppdms01 = ppdms01.replace(0, np.nan)
print('ppdms01 shape:', ppdms01.shape)

ppdms01 shape: (11875, 1)


In [5]:
### Medical
medsy01 = baseline(read_file(path+'medsy01.txt'))
a = pd.Series(medsy01.columns)
cols = a[a.str.contains('_24')].values
medsy01 = medsy01[cols]
medsy01 = medsy01.replace(999.0, 0)
medsy01 = medsy01.fillna(0)
f = lambda x: 1 if x>=1 else 0
cols = medsy01.columns
a = pd.Series(medsy01.columns)
rx_cols = a[a.str.contains('rx')].values
otc_cols = a[a.str.contains('otc')].values
medsy01['rx_24']=medsy01[rx_cols].sum(axis=1).apply(f)
medsy01['otc_24']=medsy01[otc_cols].sum(axis=1).apply(f)
medsy01['caff_24'] = medsy01['caff_24'].astype(int)
medsy01 = medsy01[['rx_24', 'otc_24', 'caff_24']]
print('medsy01 shape:', medsy01.shape)

medsy01 shape: (11875, 3)


In [6]:
### Socioecomonic Factors
pdem02 = baseline(read_file(path+'pdem02.txt'))
cols = ['demo_brthdat_v2','demo_ed_v2',
        'demo_race_a_p___10','demo_race_a_p___11','demo_race_a_p___12',
        'demo_race_a_p___13','demo_race_a_p___14','demo_race_a_p___15',
        'demo_race_a_p___16','demo_race_a_p___17','demo_race_a_p___18',
        'demo_race_a_p___19','demo_race_a_p___20','demo_race_a_p___21',
        'demo_race_a_p___22','demo_race_a_p___23', 
        'demo_prnt_marital_v2','demo_prnt_ed_v2','demo_prnt_income_v2',
        'demo_prnt_prtnr_v2','demo_prtnr_ed_v2','demo_comb_income_v2']
pdem02['race_white'] = pdem02['demo_race_a_p___10']
pdem02['race_mixed'] = pdem02[['demo_race_a_p___11','demo_race_a_p___12','demo_race_a_p___13',
                               'demo_race_a_p___14','demo_race_a_p___15','demo_race_a_p___16',
                               'demo_race_a_p___17','demo_race_a_p___18','demo_race_a_p___19',
                               'demo_race_a_p___20','demo_race_a_p___21','demo_race_a_p___22',
                               'demo_race_a_p___23']].sum(axis=1)
pdem02['race_mixed'] = pdem02['race_mixed'].apply(f)
pdem02['demo_prnt_ed_v2'] = pdem02['demo_prnt_ed_v2'].replace(999, 0) 
pdem02['demo_prnt_ed_v2'] = pdem02['demo_prnt_ed_v2'].replace(777, 0)
pdem02['demo_prnt_ed_v2'] = pdem02['demo_prnt_ed_v2'].replace(np.nan, 0)
pdem02['demo_prtnr_ed_v2'] = pdem02['demo_prtnr_ed_v2'].replace(999, 0) 
pdem02['demo_prtnr_ed_v2'] = pdem02['demo_prtnr_ed_v2'].replace(777, 0)
pdem02['demo_prtnr_ed_v2'] = pdem02['demo_prtnr_ed_v2'].replace(np.nan, 0)
pdem02['parent_edu_max'] = pdem02[['demo_prnt_ed_v2','demo_prtnr_ed_v2']].max(axis=1)
pdem02['parent_edu_max'] = pdem02['parent_edu_max'].replace(0, np.nan)
pdem02 = pdem02[['demo_brthdat_v2','demo_ed_v2','race_white','race_mixed',
                 'demo_prnt_marital_v2','parent_edu_max','demo_prnt_prtnr_v2',
                 'demo_comb_income_v2']]
pdem02 = pdem02.replace(999.0, np.nan)
pdem02 = pdem02.replace(777.0, np.nan)
print('pdem02 shape:', pdem02.shape)

pdem02 shape: (11875, 8)


In [7]:
###  Sleeping, Screen time, and Family Environment
sds01 = baseline(read_file(path+'abcd_sds01.txt'))[['sleepdisturb1_p']]
sds01 = sds01.reset_index().drop_duplicates().set_index('subjectkey')
print('sds01 shape:', sds01.shape)

stq01 = baseline(read_file(path+'stq01.txt'))[['screentime2_p_hours']]
print('stq01 shape:', stq01.shape)

fes02 = baseline(read_file(path+'fes02.txt'))[['fam_enviro1_p','fam_enviro2r_p', 'fam_enviro3_p',
                                                'fam_enviro4r_p','fam_enviro5_p', 'fam_enviro6_p',                                                
                                                'fam_enviro7r_p','fam_enviro8_p', 'fam_enviro9r_p']]
fes02['fam_enviro_sum'] = fes02.sum(axis=1)
fes02 = fes02[['fam_enviro_sum']].astype('int')
fes02 = fes02.reset_index().drop_duplicates().set_index('subjectkey')
print('fes02 shape:', fes02.shape)

sds01 shape: (11875, 1)
stq01 shape: (11875, 1)
fes02 shape: (11875, 1)


In [8]:
### Gender
lt01 = baseline(read_file(path + 'abcd_lt01.txt'))[['gender']]
f = lambda x: 1 if x=='F' else x
m = lambda x: 0 if x=='M' else x
lt01['gender'] = lt01['gender'].apply(m).apply(f)
print('lt01 shape:', lt01.shape)

lt01 shape: (11875, 1)


In [9]:
### Family Depression
fhxp102 = baseline(read_file(path+'fhxp102.txt'))[['fam_history_q6a_depression', 'fam_history_q6d_depression']]
fhxp102 = fhxp102.replace(np.nan, 0)
fhxp102 = fhxp102.replace(999.0, np.nan)
fhxp102 = fhxp102.reset_index().drop_duplicates().set_index('subjectkey')
fhxp102['fam_history_depression'] = np.logical_or(fhxp102['fam_history_q6a_depression'], fhxp102['fam_history_q6d_depression'])
fhxp102 = fhxp102.drop(['fam_history_q6a_depression', 'fam_history_q6d_depression'], axis=1).astype('int')
print('fhxp102 shape:', fhxp102.shape)

fhxp102 shape: (11875, 1)


In [10]:
### original cov
covariates = pd.concat([fhxp102,lt01,fes02,stq01,sds01,pdem02,medsy01,ppdms01,ant01],axis=1)

covariates = covariates.astype('float', errors='ignore')
print('original covariates shape:', covariates.shape)

original covariates shape: (11875, 18)


In [11]:
##  Life events

# gish_y_ss_m_sum_nm & pps_y_ss_bother_sum_nm help boost up 8% of variance in bpm but same in cbcl
mhy = followup(read_file(path+'abcd_mhy02.txt'))
a = pd.Series(mhy.columns)
cols = a[a.str.contains('affected_*')].values
mhy = mhy[['ple_y_ss_affected_bad_sum','ple_y_ss_affected_good_sum',
           'pps_y_ss_bother_sum_nm','gish_y_ss_m_sum_nm'
          ]]
mhy.dropna(axis=1, how='all')
print('mhy shape:', mhy.shape)

mhy shape: (11235, 4)


In [12]:
##  Parent Acceptance
sscey01= baseline(read_file(path+'abcd_sscey01.txt'))
a = pd.Series(sscey01.columns)

cols = a[a.str.contains('_ss_')].values
sscey01= sscey01[['srpf_y_ss_iiss','crpbi_y_ss_caregiver','psb_y_ss_mean','fes_y_ss_fc_pr','pmq_y_ss_mean',]]
sscey01.dropna(axis=1, how='all')
print('sscey01 shape:', sscey01.shape)
sscey01

sscey01 shape: (11878, 5)


,srpf_y_ss_iiss,crpbi_y_ss_caregiver,psb_y_ss_mean,fes_y_ss_fc_pr,pmq_y_ss_mean
subjectkey,,,,,
NDAR_INV005V6D2C,14.0,NaN,2.000000,0.0,4.6
NDAR_INV00BD7VDC,13.0,2.4,1.666667,0.0,4.4
NDAR_INV00LJVZK2,13.0,2.4,1.666667,2.0,4.8
NDAR_INV014RTM1V,14.0,3.0,2.000000,0.0,5.0
NDAR_INV0182J779,15.0,2.8,2.000000,0.0,4.2
...,...,...,...,...,...
NDAR_INVZWWDT1TG,13.0,3.0,2.000000,1.0,4.8
NDAR_INVZXL47HRG,15.0,3.0,2.000000,0.0,4.8
NDAR_INVZYTK0K1Y,15.0,2.4,1.333333,3.0,4.2


In [13]:
## Parent_monitor
pmq = baseline(read_file(path+'pmq01.txt'))
a = pd.Series(pmq.columns)
cols = a[a.str.contains('parent_monitor')].values
pmq = pmq[cols]
print('Parent_monitor shape:', pmq.shape)
pmq

Parent_monitor shape: (11875, 5)


,parent_monitor_q1_y,parent_monitor_q2_y,parent_monitor_q3_y,parent_monitor_q4_y,parent_monitor_q5_y
subjectkey,,,,,
NDAR_INV0A87RKWD,5.0,5.0,5.0,5.0,5.0
NDAR_INV0AYJMFMW,5.0,3.0,4.0,5.0,5.0
NDAR_INV0BEHBEJ3,5.0,4.0,5.0,4.0,5.0
NDAR_INV0BEPJHU1,4.0,5.0,5.0,1.0,3.0
NDAR_INV0CCVJ39W,5.0,5.0,5.0,5.0,5.0
...,...,...,...,...,...
NDAR_INVZZPKBDAC,5.0,5.0,5.0,3.0,4.0
NDAR_INVZZZP87KR,5.0,5.0,5.0,4.0,5.0
NDAR_INVF0C17HWX,5.0,4.0,4.0,3.0,3.0


In [14]:
### Adult Self Report
asrs01 =baseline(read_file(path+'abcd_asrs01.txt'))[['asr_scr_attention_r','asr_scr_external_r','asr_scr_internal_r']]
print('asrs01 shape:', asrs01.shape)

asrs01 shape: (11878, 3)


In [15]:
extra=pd.concat([asrs01,sscey01,mhy],axis=1).dropna()
print('extra shape:', extra.shape)

extra shape: (10381, 12)


In [16]:
final=pd.concat([extra,covariates],axis=1).dropna()
final = final.astype('float', errors='ignore')
print('final shape:', final.shape)

final shape: (4179, 30)


### Loading Structural MRI

In [17]:
def load_from_rds(names, eventname='baseline_year_1_arm_1'):
    
    data = pd.read_csv('D:\BLUEMOON\\nda_data\\nda_rds_201.csv',
                       usecols=['src_subject_id', 'eventname'] + names,
                       na_values=['777', 999, '999', 777])
    
    data = data.loc[data[data['eventname'] == eventname].index]
    data = data.set_index('src_subject_id')
    data = data.drop('eventname', axis=1)
    
    return data

In [18]:
all_cols = list(pd.read_csv('D:\BLUEMOON\\nda_data\\nda_rds_201.csv', nrows=0))
all_cols[:10]

['subjectid',
 'src_subject_id',
 'eventname',
 'anthro_1_height_in',
 'anthro_2_height_in',
 'anthro_3_height_in',
 'anthro_height_calc',
 'anthro_weight_cast',
 'anthro_weight_a_location',
 'anthro_weight1_lb']

In [19]:
measures = ['smri_thick','smri_area','smri_vol_subcort']
                 
parcs = ['.destrieux', '_subcort.aseg']

data_cols = [col for col in all_cols
             if any([ct for ct in measures if ct in col])
             and any([p for p in parcs if p in col])] 
len(data_cols)


346

In [20]:
# Load the actual data from the saved csv
df = load_from_rds(data_cols)

,smri_thick_cort.destrieux_g.and.s.frontomargin.lh,smri_thick_cort.destrieux_g.and.s.occipital.inf.lh,smri_thick_cort.destrieux_g.and.s.paracentral.lh,smri_thick_cort.destrieux_g.and.s.subcentral.lh,smri_thick_cort.destrieux_g.and.s.transv.frontopol.lh,smri_thick_cort.destrieux_g.and.s.cingul.ant.lh,smri_thick_cort.destrieux_g.and.s.cingul.mid.ant.lh,smri_thick_cort.destrieux_g.and.s.cingul.mid.post.lh,smri_thick_cort.destrieux_g.cingul.post.dorsal.lh,smri_thick_cort.destrieux_g.cingul.post.ventral.lh,...,smri_vol_subcort.aseg_cc.mid.posterior,smri_vol_subcort.aseg_cc.central,smri_vol_subcort.aseg_cc.mid.anterior,smri_vol_subcort.aseg_cc.anterior,smri_vol_subcort.aseg_wholebrain,smri_vol_subcort.aseg_latventricles,smri_vol_subcort.aseg_allventricles,smri_vol_subcort.aseg_intracranialvolume,smri_vol_subcort.aseg_supratentorialvolume,smri_vol_subcort.aseg_subcorticalgrayvolume
src_subject_id,,,,,,,,,,,,,,,,,,,,,
NDAR_INV003RTV85,2.643,2.597,2.682,3.016,2.776,3.012,2.894,2.874,2.865,2.350,...,368.5,403.0,396.9,546.9,1.099494e+06,4693.2,6299.4,1.354788e+06,9.738411e+05,54112.0
NDAR_INV005V6D2C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NDAR_INV007W6H7B,2.798,2.635,2.620,2.963,3.038,2.948,2.966,2.728,3.263,1.882,...,352.1,371.1,336.7,684.0,1.444690e+06,13426.2,18810.3,1.703982e+06,1.290405e+06,71188.0
NDAR_INV00BD7VDC,2.570,3.008,2.771,3.116,2.753,3.137,3.222,3.062,3.315,3.065,...,365.3,357.6,432.3,720.6,1.421171e+06,8375.3,11828.6,1.679526e+06,1.283405e+06,61985.0
NDAR_INV00CY2MDM,2.589,2.495,2.732,2.982,2.979,2.953,2.732,2.819,2.908,2.967,...,463.8,414.5,398.6,824.5,1.186497e+06,19138.9,21191.9,1.561216e+06,1.072113e+06,61855.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NDAR_INVZZNX6W2P,2.604,2.839,2.642,3.017,2.990,3.119,3.014,2.871,3.240,2.254,...,476.3,338.0,366.9,761.9,1.139532e+06,11129.1,14259.9,1.480336e+06,1.001272e+06,59550.0
NDAR_INVZZPKBDAC,2.665,2.915,2.661,3.114,2.968,3.167,3.058,2.976,3.355,2.168,...,357.2,429.8,367.8,609.3,1.134203e+06,2855.1,4925.1,1.470497e+06,9.897016e+05,61090.0
NDAR_INVZZZ2ALR6,2.517,2.743,2.607,3.210,2.847,2.954,2.965,2.846,3.211,2.741,...,499.8,436.3,472.6,855.6,1.301402e+06,8278.4,10434.1,1.455727e+06,1.172208e+06,64413.0


In [ ]:
# Cast from a dataframe to BPt Dataset class
data = bp.Dataset(df)
    
# Obsificate subject ID for public example
#data.index = list(range(len(data)))

# Set optional verbosity of
data.verbose = 1

In [ ]:
### Filter subjects through freesurfer qc
fsqc = baseline(read_file(path + 'freesqc01.txt'))[['fsqc_qc']]
structural=pd.concat([fsqc,data],axis=1)
print('final shape:', structural.shape)
structural['fsqc_qc'] = structural['fsqc_qc'].fillna(0)
structural = structural[structural['fsqc_qc'] == 1].drop(['fsqc_qc'], axis=1)
structural = structural.astype('float', errors='ignore')
print('structural post freesurfer qc shape:', structural.shape)

final shape: (11879, 338)
structural post freesurfer qc shape: (11265, 337)


### Loading DTI data

In [25]:
### Select Keyword(s) from DTI
measures = ['dmri_dti.fa.wm',
            #'dmri_dti.md.wm'
        #'dmri_dti.full.fa_fibe'
           ]
                 
parcs = ['.destrieux']

data_cols = [col for col in all_cols
             if any([ct for ct in measures if ct in col])
             #and any([p for p in parcs if p in col])
             ] 
len(data_cols)

222

In [26]:
data_cols

['dmri_dti.fa.wm_cort.destrieux_g.and.s.frontomargin.lh',
 'dmri_dti.fa.wm_cort.destrieux_g.and.s.occipital.inf.lh',
 'dmri_dti.fa.wm_cort.destrieux_g.and.s.paracentral.lh',
 'dmri_dti.fa.wm_cort.destrieux_g.and.s.subcentral.lh',
 'dmri_dti.fa.wm_cort.destrieux_g.and.s.transv.frontopol.lh',
 'dmri_dti.fa.wm_cort.destrieux_g.and.s.cingul.ant.lh',
 'dmri_dti.fa.wm_cort.destrieux_g.and.s.cingul.mid.ant.lh',
 'dmri_dti.fa.wm_cort.destrieux_g.and.s.cingul.mid.post.lh',
 'dmri_dti.fa.wm_cort.destrieux_g.cingul.post.dorsal.lh',
 'dmri_dti.fa.wm_cort.destrieux_g.cingul.post.ventral.lh',
 'dmri_dti.fa.wm_cort.destrieux_g.cuneus.lh',
 'dmri_dti.fa.wm_cort.destrieux_g.front.inf.opercular.lh',
 'dmri_dti.fa.wm_cort.destrieux_g.front.inf.orbital.lh',
 'dmri_dti.fa.wm_cort.destrieux_g.front.inf.triangul.lh',
 'dmri_dti.fa.wm_cort.destrieux_g.front.middle.lh',
 'dmri_dti.fa.wm_cort.destrieux_g.front.sup.lh',
 'dmri_dti.fa.wm_cort.destrieux_g.ins.lg.and.s.cent.ins.lh',
 'dmri_dti.fa.wm_cort.destrieux_

In [27]:
fiber = load_from_rds(data_cols)
fiber= bp.Dataset(fiber)
fiber.verbose = 1
print('fiber shape:', fiber.shape)

fiber shape: (11875, 222)


In [28]:
dti = load_from_rds(data_cols)
dti= bp.Dataset(dti)
dti.verbose = 1
print('dti shape:', dti.shape)

dti shape: (11875, 222)


In [29]:
dti_combo=pd.concat([dti,fiber],axis=1)

### Loading Resting State fMRI

In [ ]:
measures = ['tfmri_nback_all_2.back_',]
                 
parcs = ['.destrieux']

data_cols = [col for col in all_cols
             if any([ct for ct in measures if ct in col])
             and any([p for p in parcs if p in col])
                    ] 
nback = load_from_rds(data_cols)
nback = bp.Dataset(nback)
nback .verbose = 1
print('nback shape:', nback.shape)


nback shape: (11875, 296)


In [31]:
measures = ['rsfmri_var_cort','rsfmri_var_subcort']
                 
parcs = ['gordon']

data_cols = [col for col in all_cols
             if any([ct for ct in measures if ct in col])
             and any([p for p in parcs if p in col])
                    ] 
rsfc = load_from_rds(data_cols)
rsfc = bp.Dataset(rsfc)
rsfc .verbose = 1
print('rsfc shape:', rsfc.shape)

rsfc shape: (11875, 333)


In [33]:
### QC section from the paper
mrirstv02 = baseline(read_file(path+'abcd_mrirstv02.txt'))[['rsfmri_var_meanmotion', 'rsfmri_var_ntpoints']]
print('mrirstv02 shape:', mrirstv02.shape)

betnet02 = baseline(read_file(path+'abcd_betnet02.txt')).iloc[:,21:-2]
features = []
for col in betnet02.columns:
    elements = col.split('_')
    if 'n' in elements:
        continue
    else:
        features.append(col)
betnet02 = betnet02[features]
print('betnet02 shape:', betnet02.shape)

subset = ['aglh', 'agrh', 'hplh', 'hprh', 'aalh', 'aarh', 'ptlh', 'ptrh', 'cdelh', 'cderh']
mrirscor02 = baseline(read_file(path + 'mrirscor02.txt'))
features = []
for col in list(mrirscor02.columns):
    elements = col.split('_')
    if('none' in elements):
        continue
    for region in subset:
        if(region in elements):
            features.append(col)
mrirscor02 = mrirscor02[features]

d1 = mrirstv02.merge(betnet02, on='subjectkey', how='outer', validate='1:1')
d2 = d1.merge(mrirscor02, on='subjectkey', how='outer', validate='1:1')
print('functional union shape:', d2.shape)
#d3 = d2.merge(fsqc, on='subjectkey', how='inner', validate='1:1')
#d3['fsqc_qc'] = d3['fsqc_qc'].fillna(0)
functional = d2
print('final shape:', functional.shape)

functional = functional[functional['fsqc_qc'] == 1].drop(['fsqc_qc'], axis=1)
print('functional post freesurfer qc shape:', functional.shape)

exclude_subjects = set()
#fmriqc01 = baseline(read_file(path+'fmriqc01.txt'))[['fmri_postqc_b0warp', 'fmri_postqc_imgqual', 'fmri_postqc_cutoff']]
#SK = set(fmriqc01.index.values)
#imputer = SimpleImputer(strategy='constant')
#fmriqc01[:] = imputer.fit_transform(fmriqc01)
#sk = set(fmriqc01[(fmriqc01['fmri_postqc_b0warp']<=1.5) & (fmriqc01['fmri_postqc_imgqual']<=1.5) & (fmriqc01['fmri_postqc_cutoff']<=1.5)].index.values)
#excluded_subjects = SK - sk
#exclude_subjects = exclude_subjects.union(excluded_subjects)

mrirstv02 = baseline(read_file(path+'abcd_mrirstv02.txt'))[['rsfmri_var_meanmotion', 'rsfmri_var_ntpoints']]
SK = set(mrirstv02.index.values)
sk = set(mrirstv02[mrirstv02['rsfmri_var_ntpoints']>375].index.values)
excluded_subjects = SK - sk
exclude_subjects = exclude_subjects.union(excluded_subjects)

indexes_to_keep = list(set(functional.index.values) - exclude_subjects)
functional = functional.loc[indexes_to_keep]
functional = functional.astype('float', errors='ignore')
print('functional post other qc filtering shape:', functional.shape)

mrirstv02 shape: (11309, 2)
betnet02 shape: (11309, 144)
functional union shape: (11347, 266)
final shape: (11347, 266)


KeyError: 'fsqc_qc'

### ML

In [35]:
# Loading Target variable

targets_to_load = ['cbcl_scr_syn_internal_r']
target_file=path+'abcd_cbcls01.txt'
#targets_to_load = ['bpm_y_scr_internal_r']
#target_file=path+'abcd_bpmt01.txt'
target =pd.read_csv(target_file,delimiter = "\t",skiprows=[1],
                    usecols=['src_subject_id'] + targets_to_load,
                    index_col='src_subject_id', na_values=['777', 999, '999', 777]
                   )
target = target[~target.index.duplicated(keep='first')]
target

,cbcl_scr_syn_internal_r
src_subject_id,
NDAR_INV59BE4FA2,0.0
NDAR_INV9EVRB30H,8.0
NDAR_INVF0C17HWX,1.0
NDAR_INV0A87RKWD,0.0
NDAR_INV0AU5R8NA,3.0
...,...
NDAR_INVZTA7JACL,9.0
NDAR_INVZVYCX57J,9.0
NDAR_INVZXC2YRV3,2.0


In [42]:
### Concatenate dataframes for brain features and generate an overall dataframe with target
brain=pd.concat([
                #rsfc,
                structural,
                #dti_combo
                ]
                ,axis=1)
overall=pd.concat([
                    final,
                    brain,
                    target],axis=1).dropna()

In [43]:
data = bp.Dataset(overall,
                  targets=targets_to_load,
                  #non_inputs='rel_family_id'
                 )

data = data.drop_subjects_by_nan(scope='target')



# Split data with family stratification
data = data.dropna()

#data = data.ordinalize(scope='rel_family_id')

#family_strat = bp.CVStrategy(groups='rel_family_id')
data

Setting NaN threshold to: 0.5


,asr_scr_attention_r,asr_scr_external_r,asr_scr_internal_r,bmi,caff_24,crpbi_y_ss_caregiver,demo_brthdat_v2,demo_comb_income_v2,demo_ed_v2,demo_prnt_marital_v2,...,smri_vol_subcort.aseg_supratentorialvolume,smri_vol_subcort.aseg_thalamus.proper.lh,smri_vol_subcort.aseg_thalamus.proper.rh,smri_vol_subcort.aseg_ventraldc.lh,smri_vol_subcort.aseg_ventraldc.rh,smri_vol_subcort.aseg_wholebrain,smri_vol_subcort.aseg_wm.hypointensities,smri_vol_subcort.aseg_wm.hypointensities.lh,smri_vol_subcort.aseg_wm.hypointensities.rh,srpf_y_ss_iiss
NDAR_INV01NAYMZH,4.0,2.0,7.0,17.636864,0.0,3.0,10.0,9.0,5.0,1.0,...,1.090852e+06,8112.4,7154.7,4046.3,3837.0,1.219416e+06,517.1,0.0,0.0,16.0
NDAR_INV02H7G2T6,0.0,1.0,1.0,20.832899,0.0,2.4,9.0,9.0,4.0,1.0,...,1.071009e+06,7593.8,6951.0,3843.2,4086.8,1.206464e+06,1637.3,0.0,0.0,14.0
NDAR_INV07RAHHYH,3.0,0.0,5.0,19.311404,0.0,3.0,9.0,10.0,4.0,1.0,...,1.003898e+06,7041.7,6893.9,3255.2,3389.2,1.127448e+06,692.0,0.0,0.0,16.0
NDAR_INV07XXFHDK,0.0,2.0,3.0,19.363688,0.0,2.2,10.0,9.0,5.0,1.0,...,1.266362e+06,9282.7,8061.9,3879.1,4146.4,1.406153e+06,801.2,0.0,0.0,13.0
NDAR_INV0BXXNBH4,2.0,2.0,4.0,18.322359,0.0,2.4,10.0,10.0,4.0,1.0,...,1.060057e+06,8553.6,7555.1,3763.7,3963.2,1.181587e+06,988.6,0.0,0.0,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NDAR_INVZXL47HRG,5.0,7.0,16.0,19.498891,0.0,3.0,9.0,7.0,4.0,3.0,...,8.949636e+05,6162.3,6091.7,2713.3,3059.8,1.001542e+06,1138.8,0.0,0.0,15.0
NDAR_INVZY8L7CT1,7.0,10.0,8.0,20.457672,0.0,3.0,10.0,7.0,5.0,1.0,...,1.369028e+06,8938.3,8307.6,4427.5,4638.6,1.522365e+06,1528.1,0.0,0.0,14.0
NDAR_INVZYB94X1J,2.0,10.0,8.0,18.010744,1.0,1.8,9.0,9.0,4.0,1.0,...,1.242586e+06,9660.0,9075.5,4744.8,5007.3,1.405908e+06,703.5,0.0,0.0,10.0
NDAR_INVZYRTFYRP,9.0,12.0,17.0,15.019452,0.0,2.2,9.0,6.0,5.0,1.0,...,8.649308e+05,7517.5,7198.7,3298.0,3482.7,9.861956e+05,547.2,0.0,0.0,11.0


In [44]:
## Train-test split

data = data.set_test_split(size=.2, random_state=2)
data

Performing test split on: 3398 subjects.
random_state: 2
Test split size: 0.2

Performed train/test split
Train size: 2718
Test size:  680


,asr_scr_attention_r,asr_scr_external_r,asr_scr_internal_r,bmi,caff_24,crpbi_y_ss_caregiver,demo_brthdat_v2,demo_comb_income_v2,demo_ed_v2,demo_prnt_marital_v2,...,smri_vol_subcort.aseg_supratentorialvolume,smri_vol_subcort.aseg_thalamus.proper.lh,smri_vol_subcort.aseg_thalamus.proper.rh,smri_vol_subcort.aseg_ventraldc.lh,smri_vol_subcort.aseg_ventraldc.rh,smri_vol_subcort.aseg_wholebrain,smri_vol_subcort.aseg_wm.hypointensities,smri_vol_subcort.aseg_wm.hypointensities.lh,smri_vol_subcort.aseg_wm.hypointensities.rh,srpf_y_ss_iiss
NDAR_INV01NAYMZH,4.0,2.0,7.0,17.636864,0.0,3.0,10.0,9.0,5.0,1.0,...,1.090852e+06,8112.4,7154.7,4046.3,3837.0,1.219416e+06,517.1,0.0,0.0,16.0
NDAR_INV02H7G2T6,0.0,1.0,1.0,20.832899,0.0,2.4,9.0,9.0,4.0,1.0,...,1.071009e+06,7593.8,6951.0,3843.2,4086.8,1.206464e+06,1637.3,0.0,0.0,14.0
NDAR_INV07RAHHYH,3.0,0.0,5.0,19.311404,0.0,3.0,9.0,10.0,4.0,1.0,...,1.003898e+06,7041.7,6893.9,3255.2,3389.2,1.127448e+06,692.0,0.0,0.0,16.0
NDAR_INV07XXFHDK,0.0,2.0,3.0,19.363688,0.0,2.2,10.0,9.0,5.0,1.0,...,1.266362e+06,9282.7,8061.9,3879.1,4146.4,1.406153e+06,801.2,0.0,0.0,13.0
NDAR_INV0BXXNBH4,2.0,2.0,4.0,18.322359,0.0,2.4,10.0,10.0,4.0,1.0,...,1.060057e+06,8553.6,7555.1,3763.7,3963.2,1.181587e+06,988.6,0.0,0.0,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NDAR_INVZXL47HRG,5.0,7.0,16.0,19.498891,0.0,3.0,9.0,7.0,4.0,3.0,...,8.949636e+05,6162.3,6091.7,2713.3,3059.8,1.001542e+06,1138.8,0.0,0.0,15.0
NDAR_INVZY8L7CT1,7.0,10.0,8.0,20.457672,0.0,3.0,10.0,7.0,5.0,1.0,...,1.369028e+06,8938.3,8307.6,4427.5,4638.6,1.522365e+06,1528.1,0.0,0.0,14.0
NDAR_INVZYB94X1J,2.0,10.0,8.0,18.010744,1.0,1.8,9.0,9.0,4.0,1.0,...,1.242586e+06,9660.0,9075.5,4744.8,5007.3,1.405908e+06,703.5,0.0,0.0,10.0
NDAR_INVZYRTFYRP,9.0,12.0,17.0,15.019452,0.0,2.2,9.0,6.0,5.0,1.0,...,8.649308e+05,7517.5,7198.7,3298.0,3482.7,9.861956e+05,547.2,0.0,0.0,11.0


In [46]:
# Let's define a Pipeline, first by creating a series of based objects

# Standard Scaling
scaler = bp.Scaler('standard', scope='float')
winsorize=bp.Scaler('winsorize', scope='float')
# Ridge Regression model
ridge = bp.Model('ridge', params=1)
elastic = bp.Model('elastic', params=1)
lgbm = bp.Model('lgbm',params=1)
svm= bp.Model('svm',params=1)
hgb= bp.Model('elastic',params=1) 

# Parameter search strategy,
# using different evolution and a custom nested
# CV strategy.


search = bp.ParamSearch(search_type='RandomSearch',
                        n_iter=100,
                        cv=bp.CV(splits=.5,
                                 n_repeats=5))

pipe = bp.Pipeline([scaler,winsorize,ridge], param_search=search)
pipe

Pipeline(param_search=ParamSearch(cv=CV(cv_strategy=CVStrategy(), n_repeats=5,
                                        splits=0.5),
                                  n_iter=100),
         steps=[Scaler(obj='standard'), Scaler(obj='winsorize'),
                Model(obj='ridge', params=1)])

In [47]:
# We can store some commonly used parameters in this
# ProblemSpec object. Though note, problem_type and scorer can both be automatically
# detected and set.
spec = bp.ProblemSpec(problem_type='regression',
                      scorer=['r2', 'explained_variance', 'neg_mean_squared_error'],
                      random_state=51,
                      n_jobs=8,
                      )

In [48]:
### CBCL
results = bp.evaluate(pipeline=pipe,
                      dataset=data,
                      problem_spec=spec,
                      progress_bar=False,
                      subjects='train', 
                      cv=4)

results

Predicting target = cbcl_scr_syn_internal_r
Using problem_type = regression
Using scope = all (defining a total of 376 features).
Evaluating 2718 total data points.

Training Set: (2038, 376)
Validation Set: (680, 376)
Fit fold in 127.7 seconds.
r2: 0.2053
explained_variance: 0.2060
neg_mean_squared_error: -24.16

Training Set: (2038, 376)
Validation Set: (680, 376)
Fit fold in 87.2 seconds.
r2: 0.2294
explained_variance: 0.2306
neg_mean_squared_error: -26.65

Training Set: (2039, 376)
Validation Set: (679, 376)
Fit fold in 79.1 seconds.
r2: 0.2222
explained_variance: 0.2222
neg_mean_squared_error: -20.64

Training Set: (2039, 376)
Validation Set: (679, 376)
Fit fold in 64.8 seconds.
r2: 0.1723
explained_variance: 0.1771
neg_mean_squared_error: -21.07



BPtEvaluator
------------
r2: 0.2073 ± 0.0220
explained_variance: 0.2090 ± 0.0204
neg_mean_squared_error: -23.13 ± 2.44

Saved Attributes: ['estimators', 'preds', 'timing', 'train_subjects', 'val_subjects', 'feat_names', 'ps', 'mean_scores', 'std_scores', 'weighted_mean_scores', 'scores', 'fis_', 'coef_', 'cv']

Avaliable Methods: ['get_X_transform_df', 'get_preds_dfs', 'get_fis', 'get_coef_', 'permutation_importance']

Evaluated With:
target: cbcl_scr_syn_internal_r
problem_type: regression
scope: all
subjects: train
random_state: 51
n_jobs: 8


In [49]:
## Feature Importance of the model
fis = results.get_fis(mean=True)
fis.sort_values()
len(fis)

374

In [50]:
fis = results.get_fis(mean=True)
fis.sort_values()

srpf_y_ss_iiss                                       -0.307475
smri_area_cort.destrieux_s.orbital.med.olfact.rh     -0.201879
smri_area_cort.destrieux_g.pariet.inf.angular.rh     -0.178224
smri_thick_cort.destrieux_g.and.s.occipital.inf.lh   -0.167033
smri_thick_cort.destrieux_s.temporal.sup.lh          -0.156407
                                                        ...   
rx_24                                                 0.348364
fam_history_depression                                0.352294
asr_scr_external_r                                    0.388418
asr_scr_attention_r                                   0.581153
asr_scr_internal_r                                    1.002084
Length: 374, dtype: float64

### Feature Selection

In [52]:
feat_selector = bp.FeatSelector('selector', params=1)

# This param search is responsible for optimizing the selected features from feat_selector

# We create a nested elastic net model to optimize - no particular CV should be okay
random_search = bp.ParamSearch('RandomSearch', n_iter=100)
elastic_search = bp.Model('elastic', params=1,
                          param_search=random_search)

# Put it all together in a pipeline
fs_pipe = bp.Pipeline([scaler, feat_selector, elastic_search])

In [53]:
results = bp.evaluate(pipeline=fs_pipe, dataset=data,progress_bar=False,
                      problem_spec=spec)
results

Predicting target = cbcl_scr_syn_internal_r
Using problem_type = regression
Using scope = all (defining a total of 376 features).
Evaluating 2718 total data points.

Training Set: (2174, 376)
Validation Set: (544, 376)
Fit fold in 49.7 seconds.
r2: 0.1619
explained_variance: 0.1619
neg_mean_squared_error: -23.32

Training Set: (2174, 376)
Validation Set: (544, 376)
Fit fold in 21.0 seconds.
r2: 0.2093
explained_variance: 0.2167
neg_mean_squared_error: -30.99

Training Set: (2174, 376)
Validation Set: (544, 376)
Fit fold in 20.7 seconds.
r2: 0.1546
explained_variance: 0.1547
neg_mean_squared_error: -23.28

Training Set: (2175, 376)
Validation Set: (543, 376)
Fit fold in 19.7 seconds.
r2: 0.1367
explained_variance: 0.1404
neg_mean_squared_error: -21.35

Training Set: (2175, 376)
Validation Set: (543, 376)
Fit fold in 16.0 seconds.
r2: 0.1674
explained_variance: 0.1707
neg_mean_squared_error: -22.25



BPtEvaluator
------------
r2: 0.1660 ± 0.0240
explained_variance: 0.1689 ± 0.0259
neg_mean_squared_error: -24.24 ± 3.45

Saved Attributes: ['estimators', 'preds', 'timing', 'train_subjects', 'val_subjects', 'feat_names', 'ps', 'mean_scores', 'std_scores', 'weighted_mean_scores', 'scores', 'fis_', 'coef_', 'cv']

Avaliable Methods: ['get_X_transform_df', 'get_preds_dfs', 'get_fis', 'get_coef_', 'permutation_importance']

Evaluated With:
target: cbcl_scr_syn_internal_r
problem_type: regression
scope: all
subjects: train
random_state: 51
n_jobs: 8


In [54]:
fis = results.get_fis(mean=True)
fis.sort_values()

pps_y_ss_bother_sum_nm                                  -0.157487
smri_thick_cort.destrieux_s.temporal.sup.lh             -0.048673
smri_area_cort.destrieux_s.oc.temp.lat.rh               -0.035381
smri_thick_cort.destrieux_lat.fis.post.lh               -0.033304
smri_area_cort.destrieux_g.pariet.inf.angular.rh        -0.030329
smri_area_cort.destrieux_g.postcentral.lh               -0.028118
smri_thick_cort.destrieux_g.insular.short.rh            -0.022528
smri_thick_cort.destrieux_s.orbital.med.olfact.lh       -0.021777
smri_area_cort.destrieux_s.circular.insula.sup.lh       -0.021359
smri_area_cort.destrieux_g.occipital.middle.rh          -0.013938
smri_area_cort.destrieux_s.oc.sup.and.transversal.rh    -0.013533
smri_area_cort.destrieux_s.precentral.sup.part.rh       -0.009950
smri_thick_cort.destrieux_s.circular.insula.ant.lh      -0.009598
smri_area_cort.destrieux_s.oc.temp.med.and.lingual.rh   -0.009159
crpbi_y_ss_caregiver                                    -0.007727
smri_vol_s

### Comparing different pipelines

In [55]:
### elastic internalizing cov only
results = bp.evaluate(pipeline=bp.Compare([fs_pipe,pipe]),
                      dataset=data,
                      problem_spec=spec,
                      subjects='train', 
                      cv=4)
results

Running Compare: Options(pipeline=Pipeline(steps=[Scaler(obj=standard), FeatSelector(obj=selector, params=1),
                Model(obj=elastic,
                      param_search=ParamSearch(cv=CV(cv_strategy=CVStrategy()),
                                               n_iter=100),
                      params=1)]))
Predicting target = cbcl_scr_syn_internal_r
Using problem_type = regression
Using scope = all (defining a total of 376 features).
Evaluating 2718 total data points.


Compare:   0%|          | 0/2 [00:00<?, ?it/s]

Folds:   0%|          | 0/4 [00:00<?, ?it/s]


Training Set: (2038, 376)
Validation Set: (680, 376)
Fit fold in 56.8 seconds.
r2: 0.1653
explained_variance: 0.1664
neg_mean_squared_error: -25.38

Training Set: (2038, 376)
Validation Set: (680, 376)
Fit fold in 15.3 seconds.
r2: 0.2113
explained_variance: 0.2154
neg_mean_squared_error: -27.27

Training Set: (2039, 376)
Validation Set: (679, 376)
Fit fold in 14.8 seconds.
r2: 0.1339
explained_variance: 0.1364
neg_mean_squared_error: -22.98

Training Set: (2039, 376)
Validation Set: (679, 376)
Fit fold in 15.1 seconds.
r2: 0.1444
explained_variance: 0.1482
neg_mean_squared_error: -21.78

Running Compare: Options(pipeline=Pipeline(param_search=ParamSearch(cv=CV(cv_strategy=CVStrategy(), n_repeats=5,
                                        splits=0.5),
                                  n_iter=100),
         steps=[Scaler(obj=standard), Scaler(obj=winsorize),
                Model(obj=ridge, params=1)]))
Predicting target = cbcl_scr_syn_internal_r
Using problem_type = regression
Using s

CompareDict({Options(pipeline=Pipeline(steps=[Scaler(obj=standard), FeatSelector(obj=selector, params=1),
                Model(obj=elastic,
                      param_search=ParamSearch(cv=CV(cv_strategy=CVStrategy()),
                                               n_iter=100),
                      params=1)])): BPtEvaluator
------------
r2: 0.1637 ± 0.0297
explained_variance: 0.1666 ± 0.0301
neg_mean_squared_error: -24.35 ± 2.13

Saved Attributes: ['estimators', 'preds', 'timing', 'train_subjects', 'val_subjects', 'feat_names', 'ps', 'mean_scores', 'std_scores', 'weighted_mean_scores', 'scores', 'fis_', 'coef_', 'cv']

Avaliable Methods: ['get_X_transform_df', 'get_preds_dfs', 'get_fis', 'get_coef_', 'permutation_importance']

Evaluated With:
target: cbcl_scr_syn_internal_r
problem_type: regression
scope: all
subjects: train
random_state: 51
n_jobs: 8

, Options(pipeline=Pipeline(param_search=ParamSearch(cv=CV(cv_strategy=CVStrategy(), n_repeats=5,
                                 

In [58]:
results.pairwise_t_stats(metric='r2')

,pipeline (1),pipeline (2),t_stat,p_val
0,"Pipeline(steps=[Scaler(obj=standard), FeatSele...",Pipeline(param_search=ParamSearch(cv=CV(cv_str...,-2.421358,0.047027
